In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yt-dlp

Note: you may need to restart the kernel to use updated packages.


In [8]:
from pygame import *
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
import time
from threading import *
from yt_dlp import YoutubeDL
import os
import subprocess

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
if not os.path.exists("downloads"):
    os.makedirs("downloads")

playlist = []
wasstarted = False
volume = 0.3
totallength = 0
quitthreading = False
slider_moved = False
playing = False

mixer.init()
player = mixer.music

In [10]:
class Music:
    def __init__(self, file):
        self.file = file
        name = file.split("/")
        self.name = name[-1]

In [11]:
def AddToList(musicobj):
    listsize = music_list.size() + 1
    music_list.insert(listsize, musicobj.name)
    playlist.append(musicobj)

In [12]:
def OpenFile():
    try:
        filename = filedialog.askopenfilenames(
            initialdir="/",
            title="Select a File",
            filetypes=(('Audio/Video Files', '*.mp3 *.mp4 *.m4a *.webm'), ('All Files', '*.*'))
        )
        if not filename:
            return

        for files in filename:
            if files.endswith((".mp4", ".m4a", ".webm")):
                audio_file = convert_to_mp3(files)
                music = Music(audio_file)
            else:
                music = Music(files)
            AddToList(music)

    except Exception as e:
        print(f"OpenFile error: {e}")

In [13]:
def download_audio_from_youtube(url):
    if not os.path.exists("downloads"):
        os.makedirs("downloads")

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'downloads/%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True
    }

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        return f"downloads/{info['title']}.mp3"

In [14]:
def convert_to_mp3(file_path):
    mp3_path = file_path.rsplit('.', 1)[0] + ".mp3"
    if not os.path.exists(mp3_path):
        try:
            subprocess.run([
                "ffmpeg", "-y", "-i", file_path, mp3_path
            ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except Exception as e:
            print(f"FFmpeg conversion error: {e}")
    return mp3_path

In [15]:
def DownloadFromYouTube():
    url = youtube_entry.get()
    if url.strip() == "":
        messagebox.showerror("Error", "Please enter a YouTube URL.")
        return
    try:
        mp3_file_path = download_audio_from_youtube(url)
        music = Music(mp3_file_path)
        AddToList(music)
        messagebox.showinfo("Success", "Downloaded and added to playlist!")
    except Exception as e:
        messagebox.showerror("Error", str(e))

In [16]:
def Play():
    global wasstarted, player, playing
    if not wasstarted:
        selected_music = music_list.curselection()
        if not selected_music:
            messagebox.showwarning("Music", "Choose a song from the list!")
            return

        index = selected_music[0]
        file_path = playlist[index].file

        if not os.path.exists(file_path):
            messagebox.showerror("File Not Found", f"Could not find the file:\n{file_path}")
            return

        try:
            player.load(file_path)
            player.play()
            player.set_volume(volume)
            UpdateTime()
            pause_button.place(x=200, y=500)
            wasstarted = True
        except Exception as e:
            messagebox.showerror("Playback Error", str(e))
    else:
        playing = True
        player.unpause()
        play_button.place_forget()
        pause_button.place(x=200, y=500)


In [17]:
def Pause():
    global playing
    playing = False
    player.pause()
    pause_button.place_forget()
    play_button.place(x=200, y=500)

In [18]:
def Stop():
    global wasstarted, playing
    if wasstarted:
        pause_button.place_forget()
        play_button.place(x=200, y=500)
        wasstarted = False
        player.stop()
        playing = False

In [19]:
def Soundd():
    soundpanel = Toplevel()
    soundpanel.geometry("200x50")
    soundpanel.resizable(False, False)
    soundslider = Scale(soundpanel, from_=0, to=100, orient=HORIZONTAL, command=SoundChanger, length=200)
    soundslider.pack()
    soundslider.set(volume * 100)

In [20]:
def SoundChanger(self):
    global volume, novolume_button, lowvolume_button, highvolume_button
    volume = int(self) / 100
    player.set_volume(volume)
    if int(self) == 0:
        novolume_button.place(x=400, y=500)
        lowvolume_button.place_forget()
        highvolume_button.place_forget()
    elif int(self) < 60:
        novolume_button.place_forget()
        highvolume_button.place_forget()
        lowvolume_button.place(x=400, y=500)
    else:
        novolume_button.place_forget()
        highvolume_button.place(x=400, y=500)
        lowvolume_button.place_forget()

In [21]:
def Forward():
    try:
        for i in music_list.curselection():
            nextmusic = i + 1
        music_list.selection_clear(0, END)
        music_list.activate(nextmusic)
        music_list.selection_set(nextmusic)
        player.load(playlist[nextmusic].file)
        player.play()
        UpdateTime()
    except:
        nextmusic = 0
        music_list.selection_clear(0, END)
        music_list.activate(nextmusic)
        music_list.selection_set(nextmusic)
        player.load(playlist[nextmusic].file)
        player.play()
        UpdateTime()

In [22]:
def Back():
    try:
        for i in music_list.curselection():
            nextmusic = i - 1
            if nextmusic < 0:
                nextmusic = music_list.size() - 1
        music_list.selection_clear(0, END)
        music_list.activate(nextmusic)
        music_list.selection_set(nextmusic)
        player.load(playlist[nextmusic].file)
        player.play()
        UpdateTime()
    except:
        nextmusic = music_list.size() - 1
        music_list.selection_clear(0, END)
        music_list.activate(nextmusic)
        music_list.selection_set(nextmusic)
        player.load(playlist[nextmusic].file)
        player.play()
        UpdateTime()

In [23]:
def PlaySelected(event=None):
    global wasstarted, player
    selected_music = music_list.curselection()
    if not selected_music:
        messagebox.showwarning("Music", "Choose a song from the list!")
        return

    index = selected_music[0]
    file_path = playlist[index].file

    if not os.path.exists(file_path):
        messagebox.showerror("File Not Found", f"Could not find the file:\n{file_path}")
        return

    try:
        player.load(file_path)
        player.play()
        player.set_volume(volume)
        pause_button.place(x=200, y=500)
        wasstarted = True
        UpdateTime()
    except Exception as e:
        messagebox.showerror("Playback Error", str(e))



In [24]:
def UpdateTime():
    global player, totallength, playing
    selected_music = music_list.curselection()[0]
    total_length = mixer.Sound.get_length(mixer.Sound(playlist[selected_music].file))
    conv_total_length = time.strftime('%M:%S', time.gmtime(total_length))
    end_song_time.config(text=conv_total_length)
    slider.config(to=int(total_length))
    pos.set(0)
    playing = True
    if not update.is_alive():
        update.start()

In [25]:
def UpdateSlider():
    global player, totallength
    while True:
        if playing:
            if slider_moved:
                pos.set(slider.get() + 1)
                conv_total_length = time.strftime('%M:%S', time.gmtime(int(slider.get())))
                current_song_time.config(text=conv_total_length)
            else:
                current_time = player.get_pos() / 1000
                conv_total_length = time.strftime('%M:%S', time.gmtime(current_time))
                current_song_time.config(text=conv_total_length)
                pos.set(int(current_time))
            slider.config(variable=pos)
            time.sleep(1)
            if quitthreading:
                break

In [26]:
def FastForward(self):
    global slider_moved
    pos.set(self)
    player.play(start=int(self))
    slider_moved = True

In [ ]:
def CloseWindow():
    global quitthreading
    quitthreading = True
    player.stop()
    master.destroy()

master = Tk()
master.title("Mp3 Player")
master.protocol("WM_DELETE_WINDOW", CloseWindow)
master.geometry("500x600")
master.resizable(False, False)
master.configure(bg="#121212")  # Dark theme
update = Thread(target=UpdateSlider)
pos = IntVar()


back_img = PhotoImage(file="backpic.PNG")
back_button = Button(master, image=back_img, command=Back, bg="#121212", bd=0)
play_img = PhotoImage(file="playpic.PNG")
play_button = Button(master, image=play_img, command=Play, bg="#121212", bd=0)
pause_img = PhotoImage(file="pausepic.PNG")
pause_button = Button(master, image=pause_img, command=Pause, bg="#121212", bd=0)
stop_img = PhotoImage(file="stoppic.PNG")
stop_button = Button(master, image=stop_img, command=Stop, bg="#121212", bd=0)
forward_img = PhotoImage(file="forwardpic.PNG")
forward_button = Button(master, image=forward_img, command=Forward, bg="#121212", bd=0)
add_img = PhotoImage(file="addmusicpic.PNG")
add_button = Button(master, image=add_img, command=OpenFile, bg="#121212", bd=0)
novolume_img = PhotoImage(file="novolumepic.PNG")
novolume_button = Button(master, image=novolume_img, command=Soundd, bg="#121212", bd=0)
lowvolume_img = PhotoImage(file="lowvolumepic.PNG")
lowvolume_button = Button(master, image=lowvolume_img, command=Soundd, bg="#121212", bd=0)
highvolume_img = PhotoImage(file="highvolumepic.PNG")
highvolume_button = Button(master, image=highvolume_img, command=Soundd, bg="#121212", bd=0)

current_song_time = Label(master, text="00:00", bg="#121212", fg="white", font=("Arial", 10))
end_song_time = Label(master, text="00:00", bg="#121212", fg="white", font=("Arial", 10))
slider = Scale(master, from_=0, to=100, orient=HORIZONTAL, command=FastForward, showvalue=False, length=315, variable=pos, bg="#282828", fg="white", troughcolor="#404040", sliderrelief=FLAT)

music_list = Listbox(master, font=("Arial", 10), width=53, height=11, selectmode=SINGLE, bg="#1e1e1e", fg="white", selectbackground="#1DB954", bd=0, highlightthickness=0)
music_list.place(x=10, y=10)
add_button.place(x=400, y=300)
back_button.place(x=100, y=500)
play_button.place(x=200, y=500)
stop_button.place(x=20, y=500)
forward_button.place(x=270, y=500)
lowvolume_button.place(x=400, y=500)
current_song_time.place(x=50, y=400)
end_song_time.place(x=410, y=400)
slider.place(x=85, y=400)

Label(master, text="YouTube URL:", font=("Arial", 10, "bold"), bg="#121212", fg="white").place(x=10, y=280)
youtube_entry = Entry(master, width=58, bg="#1e1e1e", fg="white", insertbackground="white")
youtube_entry.place(x=10, y=300)
youtube_button = Button(master, text="Download from YouTube", bg="#1DB954", fg="white", font=("Arial", 10, "bold"), command=DownloadFromYouTube, relief=FLAT, activebackground="#1ED760")
youtube_button.place(x=10, y=330)

music_list.bind("<<ListboxSelect>>", PlaySelected)
mainloop()